# (A) Dependencies

In [33]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from collections import Counter

if os.system("pip freeze | grep dataset==")!=0: os.system("pip install datasets")
if os.system("pip freeze | grep conlleval==")!=0: os.system("pip install conlleval")

from datasets import load_dataset
from conlleval import evaluate

# (B) Dataset

## Downloading the dataset
- [dataset source](https://www.clips.uantwerpen.be/conll2003/ner/)

In [34]:
#!pip install datasets
!wget https://raw.githubusercontent.com/sighsmile/conlleval/master/conlleval.py

--2023-03-24 02:18:50--  https://raw.githubusercontent.com/sighsmile/conlleval/master/conlleval.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7502 (7.3K) [text/plain]
Saving to: ‘conlleval.py’

conlleval.py        100%[===================>]   7.33K  --.-KB/s    in 0s      

2023-03-24 02:18:50 (95.4 MB/s) - ‘conlleval.py’ saved [7502/7502]



In [35]:
!ls

conlleval.py  flagged  ner_model  ner_model.zip  sample_data


## Loading the dataset


In [36]:
conll_data = load_dataset("conll2003")

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# (C) Preprocessing

## Exporting the data splits as file

In [37]:
def export_to_file(export_file_path, data):
    
    # open a file 
    with open(export_file_path, "w") as f:
        
        # iterate whole data
        for record in data:

            # unpackage the data
            ner_tags = record["ner_tags"]
            tokens = record["tokens"]
            
            # write the data as convenient notation
            if len(tokens) > 0:
                
                # notated data string
                data = (
                    str(len(tokens))
                    + "\t"
                    + "\t".join(tokens)
                    + "\t"
                    + "\t".join(map(str, ner_tags))
                    + "\n"
                )

                # writing the data into file
                f.write(data)

In [38]:
# Exporting the dataset spllits via saving as file
if not os.path.exists("data"):os.mkdir("data")
export_to_file("./data/conll_train.txt", conll_data["train"])
export_to_file("./data/conll_val.txt", conll_data["validation"])

In [39]:
# Loading the dataset splits
train_data = tf.data.TextLineDataset("./data/conll_train.txt")
val_data = tf.data.TextLineDataset("./data/conll_val.txt")

## Encoding table

In [40]:
def make_tag_lookup_table():
    # Defining the labels
    iob_labels = ["B", "I"]
    ner_labels = ["PER", "ORG", "LOC", "MISC"]
    
    # creating the unique  
    all_labels = [(label1, label2) for label2 in ner_labels for label1 in iob_labels]
    all_labels = ["-".join([a, b]) for a, b in all_labels]
    all_labels = ["[PAD]", "O"] + all_labels
    
    # Numerating the tags via converting it into dict
    lookup_dict = dict(zip(range(0, len(all_labels) + 1), all_labels))
    
    return lookup_dict


mapping = make_tag_lookup_table()
print(mapping)

{0: '[PAD]', 1: 'O', 2: 'B-PER', 3: 'I-PER', 4: 'B-ORG', 5: 'I-ORG', 6: 'B-LOC', 7: 'I-LOC', 8: 'B-MISC', 9: 'I-MISC'}


In [41]:
# Finding out the dataset tag size
all_tokens = sum(conll_data["train"]["tokens"], [])
all_tokens_array = np.array(list(map(str.lower, all_tokens)))

counter = Counter(all_tokens_array)
print(len(counter))

num_tags = len(mapping)
vocab_size = 20000

num_tags, vocab_size

21009


(10, 20000)

In [80]:
# 2 additional tokens -> unknown and masking tokens (vocab_size - 2)
vocabulary = [token for token, count in counter.most_common(vocab_size - 2)]

# The StringLook class will convert tokens to token IDs
lookup_layer = keras.layers.StringLookup(vocabulary=vocabulary)

In [82]:
vocabulary

['the',
 '.',
 ',',
 'of',
 'in',
 'to',
 'a',
 'and',
 '(',
 ')',
 '"',
 'on',
 'said',
 "'s",
 'for',
 '1',
 '-',
 'at',
 'was',
 '2',
 '0',
 '3',
 'with',
 'that',
 'he',
 'from',
 'it',
 'by',
 'is',
 ':',
 'as',
 '4',
 'had',
 'his',
 'has',
 'but',
 'an',
 'not',
 'were',
 'be',
 'after',
 'have',
 'first',
 'new',
 'who',
 'will',
 'they',
 '5',
 'two',
 'u.s.',
 'been',
 '$',
 '--',
 'their',
 'beat',
 'are',
 '6',
 'which',
 'would',
 'this',
 'up',
 'its',
 'year',
 'i',
 'last',
 'percent',
 'out',
 'we',
 'thursday',
 'one',
 'million',
 'over',
 'government',
 'wednesday',
 'police',
 '7',
 'results',
 'against',
 'second',
 'when',
 '/',
 'also',
 'tuesday',
 'three',
 'soccer',
 'president',
 'no',
 'division',
 'told',
 '10',
 'monday',
 'people',
 'about',
 'or',
 'friday',
 'league',
 'some',
 'london',
 'there',
 'world',
 'her',
 'minister',
 'under',
 'more',
 'york',
 '9',
 '1996-08-28',
 'won',
 'into',
 'state',
 'sunday',
 '8',
 'before',
 'south',
 'played',
 

## Pre-processing Implementations

In [17]:
def map_record_to_training_data(record):
    
    # Finding out the token lenght
    record = tf.strings.split(record, sep="\t")
    length = tf.strings.to_number(record[0], out_type=tf.int32)
    
    # getting the token
    tokens = record[1 : length + 1]
    
    # getting the tag and converting the convenient form
    tags = record[length + 1 :]
    tags = tf.strings.to_number(tags, out_type=tf.int64)
    tags += 1
    
    return tokens, tags

In [18]:
def lowercase_and_convert_to_ids(tokens):
    tokens = tf.strings.lower(tokens)
    return lookup_layer(tokens)

In [19]:
def tokenize_and_convert_to_ids(text):
    tokens = text.split()
    return lowercase_and_convert_to_ids(tokens)

In [20]:
# each record has a different length. for handling use `padded_batch`. 
batch_size = 32
train_dataset = (
    train_data.map(map_record_to_training_data)
    .map(lambda x, y: (lowercase_and_convert_to_ids(x), y))
    .padded_batch(batch_size)
)
val_dataset = (
    val_data.map(map_record_to_training_data)
    .map(lambda x, y: (lowercase_and_convert_to_ids(x), y))
    .padded_batch(batch_size)
)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


# (D) Modelling

## Custom Architecture

In [21]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()

        # defining  layers
        self.att = keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.ffn = keras.Sequential(
            [
                keras.layers.Dense(ff_dim, activation="relu"),
                keras.layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training=False):
        
        # Transformer block
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        
        # dense layer
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        
        return out2


In [22]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = keras.layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, inputs):
        # data prepaation
        maxlen = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        
        # Embedding
        position_embeddings = self.pos_emb(positions)
        token_embeddings = self.token_emb(inputs)
        
        # concatenation
        embeddings = token_embeddings + position_embeddings
        return embeddings


In [23]:
class NERModel(keras.Model):
    def __init__(
        self, num_tags, vocab_size, maxlen=128, embed_dim=32, num_heads=2, ff_dim=32
    ):
        super().__init__()
        self.embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
        self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
        self.dropout1 = layers.Dropout(0.1)
        self.ff = layers.Dense(ff_dim, activation="relu")
        self.dropout2 = layers.Dropout(0.1)
        self.ff_final = layers.Dense(num_tags, activation="softmax")

    def call(self, inputs, training=False):
        
        # Block-1 (Feature extraction)
        x = self.embedding_layer(inputs)
        x = self.transformer_block(x)
        x = self.dropout1(x, training=training)

        # Fully Connected
        x = self.ff(x)
        x = self.dropout2(x, training=training)
        x = self.ff_final(x)
        return x


In [ ]:
# Define a custom object for saving and loading the model
class CustomObjectScope:
    def __enter__(self):
        self.old_getattr = getattr
        setattr(tf.keras.layers, 'TransformerBlock', TransformerBlock)
        setattr(tf.keras.layers, 'TokenAndPositionEmbedding', TokenAndPositionEmbedding)
        setattr(tf.keras, 'NERModel', NERModel)
        setattr(tf.keras.losses, 'CustomNonPaddingTokenLoss', CustomNonPaddingTokenLoss)

    def __exit__(self, type, value, traceback):
        setattr(tf.keras.layers, 'TransformerBlock', self.old_getattr(tf.keras.layers, 'TransformerBlock'))
        setattr(tf.keras.layers, 'TokenAndPositionEmbedding', self.old_getattr(tf.keras.layers, 'TokenAndPositionEmbedding'))
        setattr(tf.keras, 'NERModel', self.old_getattr(tf.keras, 'NERModel'))
        setattr(tf.keras.losses, 'CustomNonPaddingTokenLoss', self.old_getattr(tf.keras.losses, 'CustomNonPaddingTokenLoss'))

## Custom Loss

In [25]:
class CustomNonPaddingTokenLoss(keras.losses.Loss):
    def __init__(self, name="custom_ner_loss"):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        # calculating the loss directly
        loss_fn = keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=keras.losses.Reduction.NONE
        )
        loss = loss_fn(y_true, y_pred)

        # filtering the masked tags
        mask = tf.cast((y_true > 0), dtype=tf.float32)
        loss = loss * mask

        # finging out the ratio
        ratio = tf.reduce_sum(loss) / tf.reduce_sum(mask)
        return ratio 

## Building a model

In [26]:
# defining the model instance
loss = CustomNonPaddingTokenLoss()
ner_model = NERModel(num_tags, vocab_size, embed_dim=32, num_heads=4, ff_dim=64)

# configurating the model
ner_model.compile(optimizer="adam", loss=loss)

## Training the model

In [27]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger, LearningRateScheduler, ReduceLROnPlateau, LambdaCallback
import numpy as np

def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.5
    epochs_drop = 10
    lrate = initial_lrate * np.power(drop, np.floor((1+epoch)/epochs_drop))
    return lrate

def print_current_lr(epoch, logs):
    print("Learning rate in epoch {} is {}".format(epoch, float(tf.keras.backend.get_value(model.optimizer.lr))))

# Temporary save paths
checkpoint_path = "TemporaryModel_ModelCheckpoint.h5"
tb_log_dir = "./logs"

In [28]:
callbacks = [
    #LearningRateScheduler(step_decay),
    TensorBoard(log_dir=tb_log_dir),
    ModelCheckpoint(filepath= checkpoint_path, save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min', verbose=1),
    EarlyStopping(monitor='val_loss', patience=10, verbose=1),
    CSVLogger(filename='training.log'),
    #LambdaCallback(on_epoch_end=print_current_lr)
]

In [31]:
# Training the model
ner_model.fit(train_dataset, callbacks=callbacks, epochs=10)

Epoch 1/10


/usr/local/lib/python3.9/dist-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


    436/Unknown - 72s 140ms/step - loss: 0.6021

439/439 [==============================] - 72s 139ms/step - loss: 0.6006
Epoch 2/10
438/439 [============================>.] - ETA: 0s - loss: 0.2531

439/439 [==============================] - 5s 12ms/step - loss: 0.2529
Epoch 3/10
436/439 [============================>.] - ETA: 0s - loss: 0.1536

...

439/439 [==============================] - 8s 18ms/step - loss: 0.0769
Epoch 7/10
435/439 [============================>.] - ETA: 0s - loss: 0.0656

439/439 [==============================] - 5s 12ms/step - loss: 0.0659
Epoch 8/10
438/439 [============================>.] - ETA: 0s - loss: 0.0572

439/439 [==============================] - 6s 14ms/step - loss: 0.0572
Epoch 9/10
436/439 [============================>.] - ETA: 0s - loss: 0.0493

439/439 [==============================] - 5s 12ms/step - loss: 0.0493
Epoch 10/10
436/439 [============================>.] - ETA: 0s - loss: 0.0424

439/439 [==============================] - 6s 14ms/step - loss: 0.0424


In [34]:
# encoding arbitrary text
ex_text = "eu rejects german call to boycott british lamb"
sample_input = tokenize_and_convert_to_ids(ex_text)
sample_input = tf.reshape(sample_input, shape=[1, -1])

# prediction
output = ner_model.predict(sample_input)

# decoding  the results
prediction = np.argmax(output, axis=-1)[0]
prediction = [mapping[i] for i in prediction]

# Showing the results
#print("eu -> B-ORG, german -> B-MISC, british -> B-MISC")
print(ex_text)
print(prediction)

1/1 [==============================] - 0s 39ms/step
eu rejects german call to boycott british lamb
['B-ORG', 'O', 'I-ORG', 'O', 'O', 'O', 'I-ORG', 'O']


## Results

In [66]:
from conlleval import evaluate
def calculate_metrics(dataset, ner_model=ner_model):

    all_true_tag_ids, all_predicted_tag_ids = [], []
    
    # iterate data
    for x, y in dataset:
        
        # prediction
        output = ner_model.predict(x)
        
        # encoding the output
        predictions = np.argmax(output, axis=-1)
        predictions = np.reshape(predictions, [-1])

        # extracting the actual value
        true_tag_ids = np.reshape(y, [-1])

        # getting the tags
        mask = (true_tag_ids > 0) & (predictions > 0)
        true_tag_ids = true_tag_ids[mask]
        predicted_tag_ids = predictions[mask]

        # collecting the results
        all_true_tag_ids.append(true_tag_ids)
        all_predicted_tag_ids.append(predicted_tag_ids)

    # Adjusting the shape of total data
    all_true_tag_ids = np.concatenate(all_true_tag_ids)
    all_predicted_tag_ids = np.concatenate(all_predicted_tag_ids)

    # processing the tags (mapping)
    predicted_tags = [mapping[tag] for tag in all_predicted_tag_ids]
    real_tags = [mapping[tag] for tag in all_true_tag_ids]


    # converting lists to strings
    predicted_tags_str = ' '.join(predicted_tags)
    real_tags_str = ' '.join(real_tags)


    # format the strings correctly
    predicted_lines = predicted_tags_str.split()
    real_lines = real_tags_str.split()

    lines = []
    for i in range(len(predicted_lines)):
        line = f"WORD O {real_lines[i]} {predicted_lines[i]}"
        lines.append(line)

    input_str = "\n".join(lines)

    # implementing the conlleval.evaluate for ner model evaluation
    #print(input_str)
    evaluate(input_str)


In [ ]:
#calculate_metrics(val_dataset)

# (E) Saving & Downloading the model

In [ ]:
#ner_model.load_weights(checkpoint_path)

### Saving the model

In [84]:
# Save the model
model_save_path = 'ner_model'
with CustomObjectScope(): ner_model.save(model_save_path)
tf.saved_model.save(lookup_layer, "lookup") #string lookup layer

In [83]:
import pickle
pickle.dump(vocabulary, open("vocabulary", 'wb'))

### Loading the model

In [91]:
# Load the model
with CustomObjectScope():
    loaded_model = keras.models.load_model(model_save_path, compile=False)
    loaded_model.compile(optimizer="adam", loss=CustomNonPaddingTokenLoss())

In [ ]:
# The StringLook class will convert tokens to token IDs
vocabulary  = pickle.load(open("vocabulary", 'rb'))
lookup_layer = keras.layers.StringLookup(vocabulary=vocabulary)

In [94]:
# encoding arbitrary text
ex_text = "eu rejects german call to boycott british lamb"
sample_input = tokenize_and_convert_to_ids(ex_text)
sample_input = tf.reshape(sample_input, shape=[1, -1])

# prediction
output = loaded_model.predict(sample_input)

# decoding  the results
prediction = np.argmax(output, axis=-1)[0]
prediction = [mapping[i] for i in prediction]

# Showing the results
#print("eu -> B-ORG, german -> B-MISC, british -> B-MISC")
print(ex_text)
print(prediction)

1/1 [==============================] - 0s 52ms/step
eu rejects german call to boycott british lamb
['B-ORG', 'O', 'I-ORG', 'O', 'O', 'O', 'I-ORG', 'O']


### Zipping the model


In [134]:
os.system(f"zip -r {model_save_path}.zip {model_save_path}" )

0

### Downloading the model

In [135]:
from google.colab import files
files.download(f"{model_save_path}.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
from google.colab import files
files.download("vocabulary")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# (G) Deployment as App

In [10]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
import os
if os.system("pip freeze | grep gradio==") != 0: os.system("pip install gradio")
import gradio as gr

In [3]:
!unzip ner_model.zip

Archive:  ner_model.zip
   creating: ner_model/
   creating: ner_model/variables/
  inflating: ner_model/variables/variables.data-00000-of-00001  
  inflating: ner_model/variables/variables.index  
  inflating: ner_model/saved_model.pb  
   creating: ner_model/assets/
  inflating: ner_model/keras_metadata.pb  
 extracting: ner_model/fingerprint.pb  


## Custom architecture 

In [4]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()

        # defining  layers
        self.att = keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.ffn = keras.Sequential(
            [
                keras.layers.Dense(ff_dim, activation="relu"),
                keras.layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training=False):
        
        # Transformer block
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        
        # dense layer
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        
        return out2


In [5]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = keras.layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, inputs):
        # data prepaation
        maxlen = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        
        # Embedding
        position_embeddings = self.pos_emb(positions)
        token_embeddings = self.token_emb(inputs)
        
        # concatenation
        embeddings = token_embeddings + position_embeddings
        return embeddings


In [6]:
class NERModel(keras.Model):
    def __init__(
        self, num_tags, vocab_size, maxlen=128, embed_dim=32, num_heads=2, ff_dim=32
    ):
        super().__init__()
        self.embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
        self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
        self.dropout1 = layers.Dropout(0.1)
        self.ff = layers.Dense(ff_dim, activation="relu")
        self.dropout2 = layers.Dropout(0.1)
        self.ff_final = layers.Dense(num_tags, activation="softmax")

    def call(self, inputs, training=False):
        
        # Block-1 (Feature extraction)
        x = self.embedding_layer(inputs)
        x = self.transformer_block(x)
        x = self.dropout1(x, training=training)

        # Fully Connected
        x = self.ff(x)
        x = self.dropout2(x, training=training)
        x = self.ff_final(x)
        return x


In [7]:
class CustomNonPaddingTokenLoss(keras.losses.Loss):
    def __init__(self, name="custom_ner_loss"):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        # calculating the loss directly
        loss_fn = keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=keras.losses.Reduction.NONE
        )
        loss = loss_fn(y_true, y_pred)

        # filtering the masked tags
        mask = tf.cast((y_true > 0), dtype=tf.float32)
        loss = loss * mask

        # finging out the ratio
        ratio = tf.reduce_sum(loss) / tf.reduce_sum(mask)
        return ratio 

In [8]:
# Define a custom object for saving and loading the model
class CustomObjectScope:
    def __enter__(self):
        self.old_getattr = getattr
        setattr(tf.keras.layers, 'TransformerBlock', TransformerBlock)
        setattr(tf.keras.layers, 'TokenAndPositionEmbedding', TokenAndPositionEmbedding)
        setattr(tf.keras, 'NERModel', NERModel)
        setattr(tf.keras.losses, 'CustomNonPaddingTokenLoss', CustomNonPaddingTokenLoss)

    def __exit__(self, type, value, traceback):
        setattr(tf.keras.layers, 'TransformerBlock', self.old_getattr(tf.keras.layers, 'TransformerBlock'))
        setattr(tf.keras.layers, 'TokenAndPositionEmbedding', self.old_getattr(tf.keras.layers, 'TokenAndPositionEmbedding'))
        setattr(tf.keras, 'NERModel', self.old_getattr(tf.keras, 'NERModel'))
        setattr(tf.keras.losses, 'CustomNonPaddingTokenLoss', self.old_getattr(tf.keras.losses, 'CustomNonPaddingTokenLoss'))

## APP (loading from file)

### Declerations & Util Funcs

In [11]:
# File Paths
model_path = 'ner_model' 
mapping = {
    0: '[PAD]',
    1: 'O', 
    2: 'B-PER', 
    3: 'I-PER', 
    4: 'B-ORG', 
    5: 'I-ORG', 
    6: 'B-LOC', 
    7: 'I-LOC', 
    8: 'B-MISC', 
    9: 'I-MISC'
}

# Load the model
with CustomObjectScope():
    model = keras.models.load_model(model_path, compile=False)
    model.compile(optimizer="adam", loss=CustomNonPaddingTokenLoss())

# lookup layer
vocabulary  = pickle.load(open("vocabulary", 'rb'))
lookup_layer = keras.layers.StringLookup(vocabulary=vocabulary)

In [12]:
def lowercase_and_convert_to_ids(tokens):
    tokens = tf.strings.lower(tokens)
    return lookup_layer(tokens)

def tokenize_and_convert_to_ids(text):
    tokens = text.split()
    return lowercase_and_convert_to_ids(tokens)

def formatting(text, prediction_labels):

    # tokenization
    prediction_tokens = text.split()

    # Generate HTML rendering with NER labels
    html = ""
    i = 0
    while i < len(prediction_tokens):
        
        # unpackaging
        token = prediction_tokens[i]
        label = prediction_labels[i]
        
        if label != "O":
            
            # setting the html
            html += "<span style='background-color: #ffff00; font-weight: bold;'>{} ({})</span>".format(token, label)
            j = i + 1
            while j < len(prediction_tokens) and prediction_labels[j] == label:
                html += " {}".format(prediction_tokens[j])
                j += 1
            html += " "
            
            i = j
        else:
            html += "{} ".format(token)
            i += 1
    return html

def predict(text):

  # encoding the text
  sample_input = tokenize_and_convert_to_ids(text)
  sample_input = tf.reshape(sample_input, shape=[1, -1])

  # prediction
  output = model.predict(sample_input)

  # decoding  the results
  prediction = np.argmax(output, axis=-1)[0]
  prediction = [mapping[i] for i in prediction]

  # html formatting for clear output 
  xml_reult = formatting(text, prediction)
  
  return xml_reult

In [13]:
predict("CRICKET	-	LEICESTERSHIRE	TAKE	OVER	AT	TOP	AFTER	INNINGS	VICTORY")

1/1 [==============================] - 3s 3s/step


"CRICKET - <span style='background-color: #ffff00; font-weight: bold;'>LEICESTERSHIRE (B-ORG)</span> TAKE OVER AT TOP AFTER INNINGS VICTORY "

In [14]:
examples = [
    "CRICKET	-	LEICESTERSHIRE	TAKE	OVER	AT	TOP	AFTER	INNINGS	VICTORY",
    "Result	and	close	of	play	scores	in	English	county	championship	matches	on	Friday",
    "Adams	and	Platt	are	both	injured	and	will	miss	England	's	opening	World	Cup	qualifier	against	Moldova	on	Sunday",
    "Lying	three	points	behind	Alania	and	two	behind	Dynamo	Moscow	,	the	Volgograd	side	have	a	game	in	hand	over	the	leaders	and	two	over	the	Moscow	club"
]

### Launching the app

In [15]:
# Interface
if_params = {
    "fn": predict ,
    "inputs": gr.inputs.Textbox(lines=5, label="Text"),
    "outputs": gr.outputs.HTML(label="NER Output"),
    "title": "Custom Named Entity Recognition Model",
    "description": "Enter a sentence to identify named entities in the text",
    "examples":examples
}

gr.Interface(**if_params).launch()

/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>